# 8.5. Implementation of Recurrent Neural Networks from Scratch

In this section we will implement an RNN from scratch for a character-level language model, according to our descriptions in Section 8.4. Such a model will be trained on H. G. Wells’ The Time Machine. As before, we start by reading the dataset first, which is introduced in Section 8.3.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
use d2l::Vocab;
use d2l::SeqDataLoader;
IPerl->load_plugin('Chart::Plotly'); # Jupyter
#import Chart::Plotly 'show_plot'; # localmente

In [2]:
my ($batch_size, $num_steps) = (32, 35);
my ($train_iter, $vocab) = d2l->load_data_time_machine(batch_size      => $batch_size, 
                                                       num_steps       => $num_steps,
                                                       use_random_iter => 1);

CODE(0xb197140)Vocab=HASH(0xb16a628)

In [3]:
print "vocab:", dump $vocab;

vocab:bless({
  _token_freqs => [
                    [" ", 32926],
                    ["e", 19781],
                    ["t", 15155],
                    ["a", 12752],
                    ["i", 11312],
                    ["o", 11157],
                    ["n", 11012],
                    ["s", 9313],
                    ["r", 8895],
                    ["h", 8841],
                    ["d", 6868],
                    ["l", 6658],
                    ["m", 4420],
                    ["u", 4328],
                    ["c", 4049],
                    ["f", 3749],
                    ["w", 3524],
                    ["g", 3512],
                    ["y", 3024],
                    ["p", 2867],
                    ["b", 2171],
                    ["v", 1435],
                    ["k", 1231],
                    ["x", 282],
                    ["j", 187],
                    ["z", 108],
                    ["q", 106],
                  ],
  idx_to_token => [
                    "<unk>",
  

1

In [4]:
# Iterator
while ( my $minibatch = $train_iter->() ) {
  print "\n\nX:", dump($minibatch->{data}->asarray), "\n";
  print "\n\nY:", dump($minibatch->{label}->asarray), "\n\n";
  last;
}



X:[
  [
    3,
    2,
    9,
    16,
    9,
    6,
    13,
    1,
    3,
    10,
    2,
    1,
    21,
    2,
    18,
    5,
    7,
    7,
    5,
    7,
    18,
    1,
    3,
    6,
    1,
    3,
    10,
    2,
    1,
    2,
    7,
    11,
    1,
    6,
    16,
  ],
  [
    7,
    1,
    13,
    4,
    23,
    5,
    7,
    18,
    1,
    8,
    20,
    4,
    8,
    13,
    6,
    11,
    5,
    15,
    1,
    2,
    16,
    16,
    6,
    9,
    3,
    8,
    1,
    3,
    6,
    1,
    9,
    2,
    12,
    5,
    18,
  ],
  [
    1,
    6,
    7,
    1,
    4,
    1,
    16,
    12,
    4,
    3,
    1,
    8,
    14,
    9,
    16,
    4,
    15,
    2,
    1,
    17,
    10,
    5,
    15,
    10,
    1,
    10,
    4,
    8,
    1,
    6,
    7,
    12,
    19,
    1,
    3,
  ],
  [
    7,
    18,
    4,
    18,
    6,
    1,
    5,
    1,
    10,
    4,
    11,
    1,
    4,
    1,
    22,
    4,
    18,
    14,
    2,
    1,
    5,
    7,
    23,
    12,
    5,
    7,
    1

## 8.5.1. One-Hot Encoding

Recall that each token is represented as a numerical index in `train_iter`.
Feeding these indices directly to a neural network might make it hard to
learn.
We often represent each token as a more expressive feature vector.
The easiest representation is called *one-hot encoding*,
which is introduced
in Section 3.4.1.

In a nutshell, we map each index to a different unit vector: assume that the number of different tokens in the vocabulary is $N$ (`len(vocab)`) and the token indices range from 0 to $N-1$.
If the index of a token is the integer $i$, then we create a vector of all 0s with a length of $N$ and set the element at position $i$ to 1.
This vector is the one-hot vector of the original token. The one-hot vectors with indices 0 and 2 are shown below.


In [16]:
print $vocab->len;
print "\nTokens to ids: ", dump $vocab->getitem([ " ", "b", "u", "t", " ", "s", "o", "m", "e", " " ]);
print "\nIds to tokens: ", dump $vocab->to_tokens([ 1, 21, 14, 3, 1, 8, 6, 13, 2, 1 ]);

28
Tokens to ids: [1, 21, 14, 3, 1, 8, 6, 13, 2, 1]
Ids to tokens: [" ", "b", "u", "t", " ", "s", "o", "m", "e", " "]

1